In [8]:
import pandas as pd
!pip install transformers tqdm


In [9]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import os
import torch
print("CUDA available:", torch.cuda.is_available())
# Load model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Label map
label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

# Load user data
users = pd.read_csv("/content/Users.csv")

# Load or initialize sentiment data
sentiment_file = "/content/UsersWithSentiment.csv"
if os.path.exists(sentiment_file):
    users_with_sentiment = pd.read_csv(sentiment_file)
else:
    users_with_sentiment = users.copy()
    users_with_sentiment["SentimentLabel"] = pd.NA
    users_with_sentiment["SentimentScore"] = pd.NA

# Process in batches until done
while True:
    mask = users_with_sentiment["SentimentScore"].isna() & users_with_sentiment["AboutMe"].notna() & (users_with_sentiment["AboutMe"].str.strip() != "")
    valid_users = users_with_sentiment[mask].head(1000).copy()

    if valid_users.empty:
        print("All users already processed. Nothing to do.")
        break

    print(f"Processing {len(valid_users)} users...")

    texts = valid_users["AboutMe"].astype(str).tolist()
    labels = []
    scores = []

    for i in tqdm(range(0, len(texts), 100), desc="Progress"):
        batch = texts[i:i+100]
        results = sentiment_model(batch, truncation=True, max_length=512)
        for r in results:
            labels.append(label_map[r["label"]])
            scores.append(r["score"])

    valid_users = valid_users.iloc[:len(labels)].copy()
    valid_users["SentimentLabel"] = labels
    valid_users["SentimentScore"] = scores

    users_with_sentiment.update(valid_users[["Id", "SentimentLabel", "SentimentScore"]])
    users_with_sentiment.to_csv(sentiment_file, index=False)
    print(f"Saved {len(valid_users)} users to {sentiment_file}\n")
